In [1]:
# !pip3 install sacrebleu

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [15]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU(force = True)

In [4]:
import malaya

model = malaya.translation.ms_en.transformer(model = 'bigbird')

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [5]:
with open('ms-en-right.test') as fopen:
    right = fopen.read().split('\n')
    
with open('ms-en-left.test') as fopen:
    left = fopen.read().split('\n')

In [19]:
from tqdm import tqdm

batch_size = 8

results = []
for i in tqdm(range(0, len(left), batch_size)):
    r_greedy = model.greedy_decoder(left[i:i + batch_size])
    results.extend(r_greedy)

100%|████████████████████████████████████████████████████████████████████████████████████████| 10950/10950 [1:31:35<00:00,  1.99it/s]


In [20]:
len(results)

87596

In [9]:
import re

def bigbird_postprocessing(string):
    return re.sub(r'([!,.?/;\'])\1+', r'\1 ', string).strip()

filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(bigbird_postprocessing(r))
        filtered_right.append(right[no])

In [16]:
refs = [filtered_right]
sys = filtered_left

In [17]:
r = bleu.corpus_score(sys, refs)

In [18]:
r.__dict__

{'name': 'BLEU',
 'score': 59.54825688811841,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '79.6/63.8/53.8/46.0 (BP = 1.000 ratio = 1.026 hyp_len = 332247 ref_len = 323967)',
 'bp': 1.0,
 'counts': [264320, 203072, 163416, 133427],
 'totals': [332247, 318079, 303914, 289798],
 'sys_len': 332247,
 'ref_len': 323967,
 'precisions': [79.55527062697331,
  63.84325906457201,
  53.770474542140214,
  46.04138054782987],
 'prec_str': '79.6/63.8/53.8/46.0',
 'ratio': 1.0255581587013491}